# SARIMA Time Series Model for Flight Delay Forecasting

This notebook implements a Seasonal Autoregressive Integrated Moving Average (SARIMA) model for forecasting flight delays using the preprocessed time series data. The SARIMA model is particularly suitable for this task as flight delay data typically exhibits both trend and seasonality components.

## Key Steps:
1. Loading the preprocessed time series data
2. Model parameter selection using ACF/PACF analysis and AIC/BIC
3. Model fitting with SARIMA
4. Model evaluation and diagnostics
5. Forecasting future flight delays
6. Visualizing and analyzing the results

In [1]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime, timedelta
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

# Configure paths dynamically using relative paths
import os.path as path

# Get the directory of the current notebook
notebook_dir = path.dirname(path.abspath('__file__'))
# Get project root (parent of notebooks directory)
project_root = path.abspath(path.join(notebook_dir, '..', '..'))

# Define paths relative to project root
TS_PROCESSED_PATH = path.join(project_root, 'data', 'processed', 'ts_ready_flights')
TS_MODEL_PATH = path.join(project_root, 'models', 'time_series')

# Create directories if they don't exist
os.makedirs(TS_MODEL_PATH, exist_ok=True)

print(f"Time Series processed data path: {TS_PROCESSED_PATH}")
print(f"Time Series model path: {TS_MODEL_PATH}")

# Set plotting style
plt.style.use('ggplot')
sns.set_palette("viridis")

# Display settings
pd.set_option('display.max_columns', None)
print("Libraries and paths configured.")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Load the preprocessed time series data
def load_ts_data(path, format='csv'):
    """
    Load time series data from CSV or pickle file
    
    Parameters:
    -----------
    path : str
        Base path to the data file
    format : str
        File format ('csv' or 'pkl')
        
    Returns:
    --------
    pandas.DataFrame
        Time series data with DatetimeIndex
    """
    file_path = f"{path}.{format}"
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    if format == 'csv':
        # Load CSV with date index
        df = pd.read_csv(file_path)
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
            df = df.set_index('date')
        elif 'index' in df.columns:
            df['index'] = pd.to_datetime(df['index'])
            df = df.set_index('index')
    elif format == 'pkl':
        # Load pickle file
        df = pd.read_pickle(file_path)
    else:
        raise ValueError(f"Unsupported format: {format}")
    
    return df

# Try to load the daily time series data
try:
    daily_ts_path = os.path.join(TS_PROCESSED_PATH, "daily_delay_ts")
    ts_daily = load_ts_data(daily_ts_path, format='csv')
    print(f"Successfully loaded daily time series data: {ts_daily.shape}")
except Exception as e:
    print(f"Error loading daily time series data: {e}")
    print("Trying to load model-ready ARIMA data instead...")
    
    try:
        # Try to load the ARIMA data directly
        with open(os.path.join(TS_PROCESSED_PATH, 'model_ready', 'arima_data.pkl'), 'rb') as f:
            arima_data = pickle.load(f)
            
        with open(os.path.join(TS_PROCESSED_PATH, 'model_ready', 'arima_exog.pkl'), 'rb') as f:
            arima_exog = pickle.load(f)
            
        print("Successfully loaded ARIMA data from pickle files")
        ts_data_loaded = True
    except Exception as e2:
        print(f"Error loading ARIMA data: {e2}")
        ts_data_loaded = False

In [ ]:
# Display the time series data
if 'ts_daily' in locals():
    # Show basic info
    print(f"Time series data shape: {ts_daily.shape}")
    print(f"Date range: {ts_daily.index.min()} to {ts_daily.index.max()}")
    print(f"Total days: {len(ts_daily)}")
    
    # Display column information
    print("\nColumns:")
    for col in ts_daily.columns:
        print(f"- {col}: {ts_daily[col].dtype}")
    
    # Display sample of the data
    print("\nSample data:")
    display(ts_daily.head())
    
    # Plot the main time series of interest
    plt.figure(figsize=(16, 8))
    plt.plot(ts_daily.index, ts_daily['avg_delay'], linewidth=1.5)
    plt.title('Daily Average Departure Delay', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Average Delay (minutes)', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
elif 'arima_data' in locals():
    # Extract basic info from ARIMA data
    print(f"ARIMA data shape: {arima_data.shape}")
    print(f"Date range: {arima_data.index.min()} to {arima_data.index.max()}")
    
    # Plot the main time series from ARIMA data
    plt.figure(figsize=(16, 8))
    plt.plot(arima_data.index, arima_data, linewidth=1.5)
    plt.title('Daily Average Departure Delay', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Average Delay (minutes)', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # If exogenous variables are available, show their info
    if arima_exog is not None:
        print(f"\nExogenous variables shape: {arima_exog.shape}")
        print("Exogenous variables columns:")
        for col in arima_exog.columns:
            print(f"- {col}")
        
        # Display a sample
        print("\nSample of exogenous data:")
        display(arima_exog.head())
else:
    print("No time series data available. Please check data loading.")

## Data Preparation for SARIMA Modeling

Before fitting the SARIMA model, we need to:
1. Check and ensure stationarity of the time series
2. Identify appropriate seasonal differencing parameters
3. Analyze ACF and PACF plots to select SARIMA parameters
4. Split the data into training and testing sets

In [ ]:
# Prepare the data for SARIMA modeling
def prepare_sarima_data(ts_data, target_col='avg_delay', test_size=0.2):
    """
    Prepare time series data for SARIMA modeling
    
    Parameters:
    -----------
    ts_data : pandas.DataFrame
        Time series data with DatetimeIndex
    target_col : str
        Column to be forecasted
    test_size : float
        Proportion of data to be used for testing
    
    Returns:
    --------
    tuple
        (train_data, test_data, exog_train, exog_test)
    """
    # If input is a Series, convert to DataFrame
    if isinstance(ts_data, pd.Series):
        # This is the case with arima_data from pickle
        ts_series = ts_data
        target_col = ts_series.name if ts_series.name else 'avg_delay'
        ts_data = pd.DataFrame({target_col: ts_series})
        ts_data.index = ts_series.index
        exog_data = arima_exog if 'arima_exog' in globals() else None
    else:
        # Case with full DataFrame
        if target_col not in ts_data.columns:
            # Default to avg_delay if the target column is not in the data
            if 'avg_delay' in ts_data.columns:
                target_col = 'avg_delay'
            else:
                raise ValueError(f"Target column '{target_col}' not found in data")
        
        # Extract the target column
        ts_series = ts_data[target_col]
        
        # Extract exogenous variables (all columns except target)
        exog_cols = [col for col in ts_data.columns if col != target_col and not col.startswith('avg_delay_lag')]
        exog_data = ts_data[exog_cols] if exog_cols else None
    
    # Split into training and testing sets
    split_idx = int(len(ts_series) * (1 - test_size))
    train_data = ts_series[:split_idx]
    test_data = ts_series[split_idx:]
    
    # Split exogenous data if available
    exog_train = None
    exog_test = None
    if exog_data is not None:
        exog_train = exog_data.iloc[:split_idx]
        exog_test = exog_data.iloc[split_idx:]
    
    return train_data, test_data, exog_train, exog_test

In [ ]:
# Check for stationarity
def check_stationarity(ts_series):
    """
    Check if a time series is stationary using the Augmented Dickey-Fuller test
    
    Parameters:
    -----------
    ts_series : pandas.Series
        Time series data
    
    Returns:
    --------
    bool
        True if stationary, False otherwise
    """
    from statsmodels.tsa.stattools import adfuller
    
    # Perform ADF test
    result = adfuller(ts_series.dropna())
    
    # Plot rolling statistics
    rolling_mean = ts_series.rolling(window=7).mean()
    rolling_std = ts_series.rolling(window=7).std()
    
    plt.figure(figsize=(16, 8))
    plt.subplot(211)
    plt.plot(ts_series.index, ts_series, label='Original')
    plt.plot(rolling_mean.index, rolling_mean, label='Rolling Mean (7-day)')
    plt.plot(rolling_std.index, rolling_std, label='Rolling Std (7-day)')
    plt.legend()
    plt.title('Rolling Mean & Standard Deviation', fontsize=16)
    plt.grid(True, alpha=0.3)
    
    plt.subplot(212)
    plt.bar(['ADF Statistic', 'p-value', '1%', '5%', '10%'], 
            [result[0], result[1], result[4]['1%'], result[4]['5%'], result[4]['10%']], 
            color=['blue', 'red', 'green', 'green', 'green'])
    plt.axhline(y=result[4]['5%'], color='green', linestyle='-', alpha=0.5)
    plt.title('Augmented Dickey-Fuller Test Results', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Print test results
    print('Augmented Dickey-Fuller Test Results:')
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.4f}')
    
    # Interpret results
    if result[1] <= 0.05:
        print("\nConclusion: Time series is stationary (reject null hypothesis)")
        return True
    else:
        print("\nConclusion: Time series is non-stationary (fail to reject null hypothesis)")
        return False

# Apply differencing if non-stationary
def difference_series(ts_series, d=1, D=0, s=0):
    """
    Apply differencing to make a time series stationary
    
    Parameters:
    -----------
    ts_series : pandas.Series
        Time series data
    d : int
        Order of non-seasonal differencing
    D : int
        Order of seasonal differencing
    s : int
        Seasonal period length
    
    Returns:
    --------
    pandas.Series
        Differenced time series
    """
    diff_series = ts_series.copy()
    
    # Apply seasonal differencing
    if D > 0 and s > 0:
        for _ in range(D):
            diff_series = diff_series.diff(s).dropna()
    
    # Apply regular differencing
    if d > 0:
        for _ in range(d):
            diff_series = diff_series.diff().dropna()
    
    return diff_series

In [ ]:
# Prepare data for SARIMA modeling
if 'ts_daily' in locals():
    # Use the full dataframe
    train_data, test_data, exog_train, exog_test = prepare_sarima_data(ts_daily)
elif 'arima_data' in locals():
    # Use the loaded ARIMA data
    train_data, test_data, exog_train, exog_test = prepare_sarima_data(arima_data)
else:
    print("No time series data available for modeling.")

# Display training and testing data
if 'train_data' in locals():
    print(f"Training data shape: {train_data.shape}")
    print(f"Training data date range: {train_data.index.min()} to {train_data.index.max()}")
    
    print(f"\nTesting data shape: {test_data.shape}")
    print(f"Testing data date range: {test_data.index.min()} to {test_data.index.max()}")
    
    # Plot train-test split
    plt.figure(figsize=(16, 8))
    plt.plot(train_data.index, train_data, label='Training Data')
    plt.plot(test_data.index, test_data, label='Testing Data', color='red')
    plt.title('Train-Test Split for SARIMA Modeling', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Average Delay (minutes)', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Check stationarity
    print("\nChecking stationarity of the training data:")
    is_stationary = check_stationarity(train_data)
    
    # If not stationary, apply differencing
    if not is_stationary:
        print("\nApplying first-order differencing:")
        diff1 = difference_series(train_data, d=1)
        is_diff1_stationary = check_stationarity(diff1)
        
        # Try seasonal differencing if still not stationary
        if not is_diff1_stationary:
            # Try with weekly seasonality (s=7)
            print("\nApplying seasonal differencing (weekly):")
            diff_seasonal = difference_series(train_data, d=0, D=1, s=7)
            is_seasonal_stationary = check_stationarity(diff_seasonal)
            
            # Try with both regular and seasonal differencing
            if not is_seasonal_stationary:
                print("\nApplying both regular and seasonal differencing:")
                diff_both = difference_series(train_data, d=1, D=1, s=7)
                is_both_stationary = check_stationarity(diff_both)

In [ ]:
# Analyze ACF and PACF plots for model parameter selection
def analyze_acf_pacf(ts_series, lags=40, diff_order=(0,0,0), seasonal=False):
    """
    Analyze ACF and PACF plots to determine SARIMA parameters
    
    Parameters:
    -----------
    ts_series : pandas.Series
        Time series data
    lags : int
        Number of lags to plot
    diff_order : tuple
        (d, D, s) for regular differencing, seasonal differencing, and seasonal period
    seasonal : bool
        Whether to analyze for seasonal patterns
    """
    d, D, s = diff_order
    
    # Apply differencing if necessary
    data = ts_series.copy()
    if D > 0 and s > 0:
        for _ in range(D):
            data = data.diff(s).dropna()
    
    if d > 0:
        for _ in range(d):
            data = data.diff().dropna()
    
    # Create the plots
    fig, axes = plt.subplots(2, 1, figsize=(16, 12))
    
    # Plot ACF
    sm.graphics.tsa.plot_acf(data.dropna(), lags=lags, alpha=0.05, 
                            title=f'Autocorrelation Function (ACF) - d={d}, D={D}, s={s}',
                            ax=axes[0])
    axes[0].grid(True, alpha=0.3)
    
    # Plot PACF
    sm.graphics.tsa.plot_pacf(data.dropna(), lags=lags, alpha=0.05, method='ols',
                             title=f'Partial Autocorrelation Function (PACF) - d={d}, D={D}, s={s}',
                             ax=axes[1])
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Analyze seasonal decomposition
def analyze_seasonal_decomposition(ts_series, period):
    """
    Decompose time series into trend, seasonal, and residual components
    
    Parameters:
    -----------
    ts_series : pandas.Series
        Time series data
    period : int
        Seasonality period
    """
    # Fill any missing values for decomposition
    ts_filled = ts_series.fillna(method='ffill')
    
    # Perform decomposition
    decomposition = seasonal_decompose(ts_filled, model='additive', period=period)
    
    # Plot decomposition
    plt.figure(figsize=(16, 12))
    plt.subplot(411)
    plt.plot(decomposition.observed)
    plt.title('Observed', fontsize=16)
    plt.grid(True, alpha=0.3)
    
    plt.subplot(412)
    plt.plot(decomposition.trend)
    plt.title('Trend', fontsize=16)
    plt.grid(True, alpha=0.3)
    
    plt.subplot(413)
    plt.plot(decomposition.seasonal)
    plt.title(f'Seasonality (Period = {period})', fontsize=16)
    plt.grid(True, alpha=0.3)
    
    plt.subplot(414)
    plt.plot(decomposition.resid)
    plt.title('Residuals', fontsize=16)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

    # Return the strength of seasonality (variance of the seasonal component relative to total)
    seasonal_strength = 1 - decomposition.resid.var() / (decomposition.seasonal + decomposition.resid).var()
    print(f"Strength of seasonality (Period = {period}): {seasonal_strength:.4f}")
    return seasonal_strength

In [ ]:
# Analyze ACF and PACF for parameter selection
if 'train_data' in locals():
    # Original series
    print("ACF and PACF of original series:")
    analyze_acf_pacf(train_data, diff_order=(0,0,0))
    
    # First differenced series
    print("ACF and PACF after first-order differencing:")
    analyze_acf_pacf(train_data, diff_order=(1,0,0))
    
    # Check weekly seasonality (s=7)
    print("ACF and PACF with weekly seasonal differencing:")
    analyze_acf_pacf(train_data, diff_order=(0,1,7))
    
    # Check both regular and seasonal differencing
    print("ACF and PACF with both regular and weekly seasonal differencing:")
    analyze_acf_pacf(train_data, diff_order=(1,1,7))
    
    # Analyze seasonal components
    print("\nAnalyzing seasonal components:")
    
    # Weekly seasonality
    weekly_strength = analyze_seasonal_decomposition(train_data, period=7)
    
    # Monthly seasonality (approx 30 days)
    monthly_strength = analyze_seasonal_decomposition(train_data, period=30)
    
    # Quarterly seasonality (approx 90 days)
    quarterly_strength = analyze_seasonal_decomposition(train_data, period=90)
    
    # Determine the strongest seasonality
    seasonalities = {
        'Weekly (7 days)': weekly_strength,
        'Monthly (30 days)': monthly_strength,
        'Quarterly (90 days)': quarterly_strength
    }
    
    strongest = max(seasonalities, key=seasonalities.get)
    print(f"\nStrongest seasonal pattern detected: {strongest} with strength {seasonalities[strongest]:.4f}")
    
    # Set seasonal period for SARIMA
    if strongest == 'Weekly (7 days)':
        seasonal_period = 7
    elif strongest == 'Monthly (30 days)':
        seasonal_period = 30
    else:
        seasonal_period = 90
    
    print(f"Using seasonal period = {seasonal_period} for SARIMA modeling")

## SARIMA Model Parameter Selection

Based on the ACF, PACF, and seasonal decomposition analysis, we can now select appropriate parameters for our SARIMA model. A SARIMA model is defined by the parameters:

SARIMA(p, d, q)(P, D, Q)s where:
- p: Order of the AR term (autoregressive)
- d: Order of differencing
- q: Order of the MA term (moving average)
- P: Seasonal order of the AR term
- D: Seasonal order of differencing
- Q: Seasonal order of the MA term
- s: Seasonal period

We'll use a grid search approach to find the optimal parameters based on AIC (Akaike Information Criterion).

In [ ]:
# Grid search for optimal SARIMA parameters
def sarima_grid_search(train_data, exog_train=None, seasonal_period=7,
                     p_range=range(0, 3), d_range=range(0, 2),
                     q_range=range(0, 3), P_range=range(0, 2),
                     D_range=range(0, 2), Q_range=range(0, 2)):
    """
    Grid search for optimal SARIMA parameters
    
    Parameters:
    -----------
    train_data : pandas.Series
        Training time series data
    exog_train : pandas.DataFrame, optional
        Exogenous variables for training
    seasonal_period : int
        Seasonality period
    p_range, d_range, q_range : range
        Ranges for non-seasonal parameters p, d, q
    P_range, D_range, Q_range : range
        Ranges for seasonal parameters P, D, Q
    
    Returns:
    --------
    dict
        Dictionary with best parameters and AIC value
    """
    best_aic = float('inf')
    best_params = None
    best_model = None
    
    total_combinations = len(p_range) * len(d_range) * len(q_range) * \
                        len(P_range) * len(D_range) * len(Q_range)
    
    print(f"Grid search over {total_combinations} parameter combinations...")
    
    # Counter for progress tracking
    counter = 0
    
    # Grid search
    for p in p_range:
        for d in d_range:
            for q in q_range:
                for P in P_range:
                    for D in D_range:
                        for Q in Q_range:
                            # Update progress
                            counter += 1
                            if counter % 10 == 0 or counter == total_combinations:
                                print(f"Progress: {counter}/{total_combinations} combinations tested")
                            
                            # Skip invalid models
                            if p == 0 and q == 0 and P == 0 and Q == 0:
                                continue
                            
                            try:
                                # Fit SARIMA model
                                model = SARIMAX(train_data,
                                              exog=exog_train,
                                              order=(p, d, q),
                                              seasonal_order=(P, D, Q, seasonal_period),
                                              enforce_stationarity=False,
                                              enforce_invertibility=False)
                                
                                results = model.fit(disp=False)
                                
                                # Calculate AIC
                                aic = results.aic
                                
                                # If better than current best, update
                                if aic < best_aic:
                                    best_aic = aic
                                    best_params = (p, d, q, P, D, Q, seasonal_period)
                                    best_model = results
                                    
                                    print(f"New best model: SARIMA{best_params} with AIC={best_aic:.4f}")
                            
                            except Exception as e:
                                # Some combinations may not converge or have other issues
                                continue
    
    if best_params is None:
        print("No valid model found. Try different parameter ranges.")
        return None
    
    # Return best parameters and model
    return {
        'order': (best_params[0], best_params[1], best_params[2]),
        'seasonal_order': (best_params[3], best_params[4], best_params[5], best_params[6]),
        'aic': best_aic,
        'model': best_model
    }

In [ ]:
# Run grid search for optimal parameters
# Note: This can be time-consuming, so we'll use restricted parameter ranges
if 'train_data' in locals():
    # Get the seasonal period determined earlier
    if 'seasonal_period' not in locals():
        seasonal_period = 7  # Default to weekly seasonality if not determined
    
    # Run grid search for optimal parameters
    # Reduced parameter space for demonstration purposes
    best_model_params = sarima_grid_search(
        train_data, 
        exog_train=exog_train, 
        seasonal_period=seasonal_period,
        p_range=range(0, 3),  # 0, 1, 2
        d_range=range(0, 2),  # 0, 1 (from stationarity test)
        q_range=range(0, 3),  # 0, 1, 2
        P_range=range(0, 2),  # 0, 1
        D_range=range(0, 2),  # 0, 1 (from seasonal differencing test)
        Q_range=range(0, 2)   # 0, 1
    )
    
    # Display the results
    if best_model_params:
        print("\nBest SARIMA Parameters:")
        print(f"Non-seasonal order (p, d, q): {best_model_params['order']}")
        print(f"Seasonal order (P, D, Q, s): {best_model_params['seasonal_order']}")
        print(f"AIC: {best_model_params['aic']:.4f}")
        
        # Store the best model
        best_sarima_model = best_model_params['model']
        
        # Summary
        print("\nModel Summary:")
        display(best_sarima_model.summary())
    else:
        print("No valid model found. Will proceed with manual parameter selection.")

In [ ]:
# Manual parameter selection if grid search fails or is too time-consuming
def fit_sarima_model(train_data, exog_train=None, order=(1,1,1), seasonal_order=(1,1,1,7)):
    """
    Fit a SARIMA model with specified parameters
    
    Parameters:
    -----------
    train_data : pandas.Series
        Training time series data
    exog_train : pandas.DataFrame, optional
        Exogenous variables for training
    order : tuple
        (p, d, q) order of the non-seasonal part
    seasonal_order : tuple
        (P, D, Q, s) order of the seasonal part
    
    Returns:
    --------
    statsmodels.tsa.statespace.sarimax.SARIMAXResults
        Fitted SARIMA model
    """
    try:
        # Fit SARIMA model
        model = SARIMAX(train_data,
                      exog=exog_train,
                      order=order,
                      seasonal_order=seasonal_order,
                      enforce_stationarity=False,
                      enforce_invertibility=False)
        
        results = model.fit(disp=False)
        
        print(f"SARIMA{order}{seasonal_order} AIC: {results.aic:.4f}")
        
        return results
    except Exception as e:
        print(f"Error fitting SARIMA{order}{seasonal_order}: {e}")
        return None

# If grid search failed or user wants to try specific parameters
if 'best_sarima_model' not in locals():
    print("Trying some common SARIMA specifications:")
    
    # Weekly seasonality models
    models = []
    
    # Try SARIMA(1,1,1)(1,1,1,7) - common for weekly patterns
    model1 = fit_sarima_model(train_data, exog_train, 
                             order=(1,1,1), 
                             seasonal_order=(1,1,1,7))
    if model1:
        models.append(('SARIMA(1,1,1)(1,1,1,7)', model1))
    
    # Try SARIMA(2,1,1)(1,1,1,7)
    model2 = fit_sarima_model(train_data, exog_train, 
                             order=(2,1,1), 
                             seasonal_order=(1,1,1,7))
    if model2:
        models.append(('SARIMA(2,1,1)(1,1,1,7)', model2))
    
    # Try SARIMA(1,1,2)(1,1,1,7)
    model3 = fit_sarima_model(train_data, exog_train, 
                             order=(1,1,2), 
                             seasonal_order=(1,1,1,7))
    if model3:
        models.append(('SARIMA(1,1,2)(1,1,1,7)', model3))
    
    # Find the best model based on AIC
    if models:
        best_model_name, best_sarima_model = min(models, key=lambda x: x[1].aic)
        print(f"\nBest manually selected model: {best_model_name} with AIC: {best_sarima_model.aic:.4f}")
        
        # Summary
        print("\nModel Summary:")
        display(best_sarima_model.summary())
    else:
        print("All manual models failed to fit. Consider trying different parameters.")

## Model Diagnostics and Validation

Now that we have fitted our SARIMA model, we need to validate it by:
1. Checking the model diagnostics (residuals)
2. Forecasting on the test set
3. Calculating error metrics (RMSE, MAE, MAPE)

In [ ]:
# Diagnostic checking
def check_model_diagnostics(model_results):
    """
    Check the diagnostics of a fitted SARIMA model
    
    Parameters:
    -----------
    model_results : statsmodels.tsa.statespace.sarimax.SARIMAXResults
        Fitted SARIMA model
    """
    # Residual diagnostics
    residuals = model_results.resid
    
    # Plot residuals
    plt.figure(figsize=(16, 12))
    
    # Residuals over time
    plt.subplot(311)
    plt.plot(residuals.index, residuals)
    plt.title('Residuals Over Time', fontsize=16)
    plt.ylabel('Residual', fontsize=12)
    plt.grid(True, alpha=0.3)
    
    # Histogram plus estimated density
    plt.subplot(312)
    sns.histplot(residuals, kde=True)
    plt.title('Histogram of Residuals', fontsize=16)
    plt.xlabel('Residual', fontsize=12)
    plt.grid(True, alpha=0.3)
    
    # QQ Plot
    plt.subplot(313)
    sm.qqplot(residuals, line='45', fit=True)
    plt.title('QQ Plot of Residuals', fontsize=16)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Check for autocorrelation in residuals
    plt.figure(figsize=(16, 8))
    
    plt.subplot(211)
    plot_acf(residuals.dropna(), lags=40, alpha=0.05, title='ACF of Residuals', ax=plt.gca())
    plt.grid(True, alpha=0.3)
    
    plt.subplot(212)
    plot_pacf(residuals.dropna(), lags=40, alpha=0.05, method='ols', title='PACF of Residuals', ax=plt.gca())
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Ljung-Box test for autocorrelation
    from statsmodels.stats.diagnostic import acorr_ljungbox
    lb_test = acorr_ljungbox(residuals.dropna(), lags=[10, 20, 30, 40])
    
    print("Ljung-Box Test for Autocorrelation in Residuals:")
    for i, lag in enumerate([10, 20, 30, 40]):
        print(f"Lag {lag}: p-value = {lb_test[1][i]:.4f}")
        if lb_test[1][i] > 0.05:
            print(f"  Fail to reject null hypothesis: No significant autocorrelation at lag {lag}")
        else:
            print(f"  Reject null hypothesis: Significant autocorrelation at lag {lag}")

# If we have a fitted model, check diagnostics
if 'best_sarima_model' in locals():
    check_model_diagnostics(best_sarima_model)
else:
    print("No fitted model available for diagnostics.")

In [ ]:
# Create forecasts and evaluate performance
def forecast_and_evaluate(model, train_data, test_data, exog_train=None, exog_test=None):
    """
    Create forecasts and evaluate model performance
    
    Parameters:
    -----------
    model : statsmodels.tsa.statespace.sarimax.SARIMAXResults
        Fitted SARIMA model
    train_data : pandas.Series
        Training time series data
    test_data : pandas.Series
        Testing time series data
    exog_train : pandas.DataFrame, optional
        Exogenous variables for training
    exog_test : pandas.DataFrame, optional
        Exogenous variables for testing
    
    Returns:
    --------
    dict
        Dictionary with forecast results and evaluation metrics
    """
    # Get forecast for test period
    forecast = model.get_forecast(steps=len(test_data), exog=exog_test)
    forecast_mean = forecast.predicted_mean
    forecast_ci = forecast.conf_int()
    
    # Plot forecast vs. actual
    plt.figure(figsize=(16, 8))
    plt.plot(train_data.index, train_data, label='Training Data')
    plt.plot(test_data.index, test_data, label='Actual Test Data')
    plt.plot(forecast_mean.index, forecast_mean, color='red', label='Forecast')
    plt.fill_between(forecast_ci.index, 
                    forecast_ci.iloc[:, 0], 
                    forecast_ci.iloc[:, 1], 
                    color='pink', alpha=0.3,
                    label='95% Confidence Interval')
    plt.title('SARIMA Forecast vs. Actual', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Average Delay (minutes)', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Calculate error metrics
    # Make sure both arrays are aligned properly
    combined = pd.DataFrame({
        'actual': test_data,
        'forecast': forecast_mean
    })
    
    # Drop any rows with missing values
    combined = combined.dropna()
    
    if len(combined) == 0:
        print("Error: No overlapping data between forecast and test data")
        return None
    
    # Calculate metrics
    mae = mean_absolute_error(combined['actual'], combined['forecast'])
    rmse = np.sqrt(mean_squared_error(combined['actual'], combined['forecast']))
    r2 = r2_score(combined['actual'], combined['forecast'])
    
    # Calculate MAPE, avoid division by zero
    mape = np.mean(np.abs((combined['actual'] - combined['forecast']) / combined['actual'])) * 100
    
    # Print metrics
    print("Forecast Evaluation Metrics:")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.4f}%")
    print(f"R-squared (R²): {r2:.4f}")
    
    # Return results
    return {
        'forecast': forecast_mean,
        'conf_int': forecast_ci,
        'metrics': {
            'mae': mae,
            'rmse': rmse,
            'mape': mape,
            'r2': r2
        }
    }

In [ ]:
# Create forecasts and evaluate performance
if 'best_sarima_model' in locals() and 'train_data' in locals() and 'test_data' in locals():
    forecast_results = forecast_and_evaluate(
        best_sarima_model, 
        train_data, 
        test_data, 
        exog_train=exog_train, 
        exog_test=exog_test
    )
else:
    print("Missing required data for forecasting and evaluation.")

## Future Forecasting

Now that we have validated our model, we can use it to forecast future flight delays. This is the main purpose of our time series model.

In [ ]:
# Forecast future values
def forecast_future(model, data, exog_data=None, steps=30):
    """
    Forecast future values using the fitted model
    
    Parameters:
    -----------
    model : statsmodels.tsa.statespace.sarimax.SARIMAXResults
        Fitted SARIMA model
    data : pandas.Series
        Complete time series data
    exog_data : pandas.DataFrame, optional
        Complete exogenous data
    steps : int
        Number of steps to forecast
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with forecasted values and confidence intervals
    """
    # Create future dates
    last_date = data.index[-1]
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=steps, freq='D')
    
    # If we have exogenous variables, we need to forecast or assume values for them
    future_exog = None
    if exog_data is not None:
        # This is a placeholder - in a real scenario, you'd need to forecast exog variables
        # or use known future values (like holidays, seasons, etc.)
        print("Note: For accurate future forecasting, you would need future exogenous variables.")
        print("For demonstration, we'll use the last values of exogenous variables.")
        
        # Use the last values as a simple approach
        last_exog_values = exog_data.iloc[-1]
        future_exog = pd.DataFrame([last_exog_values] * steps, index=future_dates)
    
    # Get forecast
    forecast = model.get_forecast(steps=steps, exog=future_exog)
    forecast_mean = forecast.predicted_mean
    forecast_ci = forecast.conf_int()
    
    # Combine into a DataFrame
    forecast_df = pd.DataFrame({
        'forecast': forecast_mean,
        'lower_ci': forecast_ci.iloc[:, 0],
        'upper_ci': forecast_ci.iloc[:, 1]
    }, index=future_dates)
    
    # Plot forecast
    plt.figure(figsize=(16, 8))
    plt.plot(data.index, data, label='Historical Data')
    plt.plot(forecast_df.index, forecast_df['forecast'], color='red', label='Forecast')
    plt.fill_between(forecast_df.index, 
                    forecast_df['lower_ci'], 
                    forecast_df['upper_ci'], 
                    color='pink', alpha=0.3,
                    label='95% Confidence Interval')
    plt.title(f'{steps}-Day Future Forecast of Flight Delays', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Average Delay (minutes)', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return forecast_df

In [ ]:
# Forecast 30 days into the future
if 'best_sarima_model' in locals():
    # Combine train and test data for full history
    if isinstance(train_data, pd.Series) and isinstance(test_data, pd.Series):
        full_data = pd.concat([train_data, test_data])
        
        # Combine exogenous data if available
        full_exog = None
        if exog_train is not None and exog_test is not None:
            full_exog = pd.concat([exog_train, exog_test])
        
        # Forecast 30 days into the future
        future_forecast = forecast_future(
            best_sarima_model, 
            full_data, 
            exog_data=full_exog, 
            steps=30
        )
        
        # Display forecast results
        print("\nForecast for the next 30 days:")
        display(future_forecast.head(10))
        
        # Save forecast to CSV
        forecast_path = os.path.join(TS_MODEL_PATH, 'sarima_forecast_30days.csv')
        future_forecast.to_csv(forecast_path)
        print(f"\nSaved forecast to {forecast_path}")
    else:
        print("Train and test data must be pandas Series for future forecasting.")
else:
    print("No fitted model available for future forecasting.")

In [ ]:
# Save the final model for future use
if 'best_sarima_model' in locals():
    # Save the model
    model_path = os.path.join(TS_MODEL_PATH, 'sarima_flight_delay_model.pkl')
    
    # Create a dictionary with model and parameters
    model_dict = {
        'model': best_sarima_model,
        'order': best_sarima_model.model.order,
        'seasonal_order': best_sarima_model.model.seasonal_order,
        'aic': best_sarima_model.aic,
        'bic': best_sarima_model.bic,
        'training_end_date': train_data.index[-1],
        'forecast_start_date': test_data.index[0]
    }
    
    # Save to pickle
    with open(model_path, 'wb') as f:
        pickle.dump(model_dict, f)
    
    print(f"Saved model to {model_path}")
    
    # Save model parameters to text file for reference
    param_path = os.path.join(TS_MODEL_PATH, 'sarima_model_params.txt')
    
    with open(param_path, 'w') as f:
        f.write("SARIMA Flight Delay Model Parameters\n")
        f.write("===================================\n\n")
        f.write(f"Non-seasonal order (p,d,q): {best_sarima_model.model.order}\n")
        f.write(f"Seasonal order (P,D,Q,s): {best_sarima_model.model.seasonal_order}\n\n")
        f.write(f"AIC: {best_sarima_model.aic:.4f}\n")
        f.write(f"BIC: {best_sarima_model.bic:.4f}\n\n")
        f.write("Model trained on data from ")
        f.write(f"{train_data.index[0]} to {train_data.index[-1]}\n\n")
        
        if 'forecast_results' in locals() and forecast_results:
            metrics = forecast_results['metrics']
            f.write("Evaluation Metrics:\n")
            f.write(f"MAE: {metrics['mae']:.4f}\n")
            f.write(f"RMSE: {metrics['rmse']:.4f}\n")
            f.write(f"MAPE: {metrics['mape']:.4f}%\n")
            f.write(f"R²: {metrics['r2']:.4f}\n")
    
    print(f"Saved model parameters to {param_path}")
else:
    print("No fitted model available to save.")

## Summary and Conclusions

In this notebook, we have:

1. Loaded and prepared time series data from the preprocessing pipeline
2. Analyzed stationarity and seasonal patterns in the flight delay data
3. Selected appropriate SARIMA parameters through grid search and diagnostic analysis
4. Fitted a SARIMA model to capture both trend and seasonal patterns in flight delays
5. Validated the model using test data and calculated performance metrics
6. Created future forecasts of flight delays

The SARIMA model is particularly well-suited for this task because flight delay data exhibits:
- Clear seasonal patterns (weekly, monthly, quarterly)
- Temporal dependence (today's delays affect tomorrow's operations)
- Need for differencing to achieve stationarity

For further improvement, we could:
- Include more external regressors (weather data, airport congestion metrics)
- Experiment with alternative models (Prophet, LSTM, hybrid approaches)
- Create separate models for different airports or carriers
- Implement ensemble methods combining multiple time series models

This model can be used by airlines and airports to anticipate periods of higher delays and allocate resources accordingly, potentially improving operational efficiency and passenger experience.